In [13]:
using NBInclude
@nbinclude("datastructs.ipynb")
@nbinclude("helpers.ipynb")

update_adj_set (generic function with 1 method)

In [14]:
function update_documentation(u_idx::Int, v_idx::Int, pq::Adjacency, excluding_pq::Adjacency, src_adjs::Vector{Adjacency}, src_gid_to_l::DefaultDict{Int, Vector{Int}})
    # update src adjs
    src_adjs[u_idx] = pq 
    src_adjs[v_idx] = excluding_pq 

    # update geneend locations # TODO 
    reassign_locs(u_idx, v_idx, pq, excluding_pq, src_gid_to_l)
end 


# find dcj operations and distance
function find_dcj_dist_ops(src_adjs::Vector{Adjacency}, tar_adj_list::Vector{Adjacency}, src_gid_to_l::DefaultDict{Int, Vector{Int}}, tar_gid_to_l::DefaultDict{Int, Vector{Int}}, src_adj_set::Set{Adjacency}, target_adj_set::Set{Adjacency})
    count = 0  
    telomere_idxs = Vector{Int}()
    updated_adj_list = src_adjs

    print("indexing through target adj list...")
    
    # for each adj {p, q} in target genome 
    for (i, adj) in pairs(tar_adj_list) 
        print("\n\n")
        show(updated_adj_list)
         
        print("\nADJ ", i, " :::: ")
        show(adj, false)
        print("\n")

        p = adj.left  
        q = adj.right  

        if p.gene == Telomere() || q.gene == Telomere()  # telomeres handled in next loop
            push!(telomere_idxs, i)  
        else 
            #  let u be adj of genome A that contains p 
            #  let v be adj of genome A that contains q 
            u, u_idx, u_adj = find_tar_ge_in_src_adjs(p, src_gid_to_l, tar_gid_to_l, src_adjs)  
            v, v_idx, v_adj = find_tar_ge_in_src_adjs(q, src_gid_to_l, tar_gid_to_l, src_adjs)  
            
            #  if u != v, replace u and v in A by {p, q} and (u\{p}) U (v\{q})
            if u_idx != v_idx             
                print("prev adj list ::: ")
                show(convert_adj_set_as_adj_list(src_adj_set), u_adj, v_adj)

                pq, excluding_pq = update_adj_set(p, q, u, v, u_idx, v_idx, src_adjs, src_adj_set)
                
                # print("updated adj set ::: ")
                # show(src_adj_set)
                print("\nupdated adj list ::: ")
                updated_adj_list = convert_adj_set_as_adj_list(src_adj_set)
                show(updated_adj_list, pq, excluding_pq)

                update_documentation(u_idx, v_idx, pq, excluding_pq, src_adjs, src_gid_to_l)
                
                # println("\n\nupdated src_adjs")
                # show(src_adjs)
                # println("\nupdated src_gid_to_location")
                # show(src_gid_to_l, id_to_char)

                count += 1 
            end   
        end
    end 
    # for each telomere {p} in genome B 
    # for idx in telomere_idxs
    #     telo_adj = tar_adj_list[idx]
    #     telo_ge = other_adjacency_end(telo_adj, Telomere())

    #     #  let u be element of genome A that contains p 
    #     u, u_idx = find_tar_ge_in_src_adjlist(p, src_gid_to_l, tar_gid_to_l, src_adjs)  

    #     #  if u is an adjacency, then replace u in A by {p} and (u\{p})
    #     if other_adjacency_end(tar_adj_list[u_idx], u) != Telomere() 
    #         # update_adj_set(p, u, u_idx,src_adjs, src_adj_set)
    #         # update_adj_list_gid_loc_dict()

    #         count += 1 
    #     end 
    # end 
    return count 
end 

src = "abcde"
target = "adcbe" 

calculate_distance(src, target)

SRC ADJ LIST
(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,e:t)(e:h,a:t)
|
v
TARGET ADJ LIST
(a:h,d:t)(d:h,c:t)(c:h,b:t)(b:h,e:t)(e:h,a:t)

*************
SRC GENE --> LOCATION DICT
e 5 => [4, 5]
b 4 => [1, 2]
d 2 => [3, 4]
c 3 => [2, 3]
a 1 => [5, 1]


TARGET GENE --> LOCATION DICT
e 5 => [4, 5]
b 4 => [3, 4]
d 2 => [1, 2]
c 3 => [2, 3]
a 1 => [5, 1]

*************

indexing through target adj list...

(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,e:t)(e:h,a:t)
ADJ 1 :::: (a:h,d:t)
prev adj list ::: (e:h,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,e:t)
updated adj list ::: (e:h,a:t)(a:h,d:t)(d:h,e:t)(b:t,c:h)(b:h,c:t)

(e:h,a:t)(a:h,d:t)(d:h,e:t)(b:t,c:h)(b:h,c:t)
ADJ 2 :::: (d:h,c:t)
prev adj list ::: (e:h,a:t)(a:h,d:t)(d:h,e:t)(b:t,c:h)(b:h,c:t)
updated adj list ::: (e:h,a:t)(a:h,d:t)(c:t,d:h)(b:t,c:h)(b:h,e:t)

(e:h,a:t)(a:h,d:t)(c:t,d:h)(b:t,c:h)(b:h,e:t)
ADJ 3 :::: (c:h,b:t)


(e:h,a:t)(a:h,d:t)(c:t,d:h)(b:t,c:h)(b:h,e:t)
ADJ 4 :::: (b:h,e:t)


(e:h,a:t)(a:h,d:t)(c:t,d:h)(b:t,c:h)(b:h,e:t)
ADJ 5 :::: (e:h,a:t)


2

In [15]:
function reassign_locs(u_idx::Int, v_idx::Int, pq::Adjacency, excluding_pq::Adjacency, gid_to_l::DefaultDict{Int, Vector{Int}})
    # want to update given: 
    # src_adjs[u_idx] = pq 
    # src_adjs[v_idx] = excluding_pq 

    idx = Ref{Int}(u_idx)
    process_adj_list_helper(pq.left, idx, gid_to_l)
    process_adj_list_helper(pq.right, idx, gid_to_l)
    
    idx = Ref{Int}(v_idx)
    process_adj_list_helper(excluding_pq.left, idx, gid_to_l)
    process_adj_list_helper(excluding_pq.right, idx, gid_to_l) 
end 

reassign_locs (generic function with 1 method)